In [14]:
"""
people with higher incomes (>50k) are more likely to accept the special promotion offer 10%
for people with lower lower income (<=50k) only 5% accept the special promotion offer

people with high income, on average, gives us 980 euro in profit
while people with lower income costs us 310 euro on average

sending a special promotion offer costs 10 euro per package

two .xlsx files are provided, "01_existing-customers.xlsx" which contains whether or not the person exceeds 50k income 
and "01_potential-customers.xlsx" which does not contain data about income
both of the files contains the following attributes:
1. age
2. workclass
3. education: highest level of education achieved by an individual
4. education-num: highest level of educationa chieved in numerical form (an integer greater than 0)
5. marital-status: marital status of an individual
6. occupation: general type of occupation of an individual
7. relationship: representa what this individual is relative to others
8. race
9. sex
10. capital-gain: capital gains for an individual (integer greater than or equal to 0)
11. capital-loss: capital losses for an individual (integer greater than or equal to 0)
12. hours-per-week: the hours an individual has reported to work per week continuous
13. native-country: country of origin for an individual
lastly, "01_existing-customers.xlsx" contains the extra attribute:
14. the label: whether or not an individual makes more than 50k a year

Solve this problem (provide the list of people to send the promotion to) and give an estimate of the profit you expect when sending the promotion to the people you selected.
The goal is to maximize the revenue

make use of the library: sklearn
first use decision trees to solve this problem
"""
import pandas as pd

# read the data
# assign column names to dataset
column_names = ["age", "workclass",	"education",	"education-num",	"marital-status", "occupation",
                "relationship",	"race",	"sex",	"capital-gain",	"capital-loss",	"hours-per-week",	"native-country",	"class"
]
existing_customers = pd.read_excel("01_existing-customers.xlsx", usecols=column_names)
# remove the last column name "class" from the list as it does not exist in the potential_customers dataset
potential_customers = pd.read_excel("01_potential-customers.xlsx", usecols=column_names[:-1])


c:\Users\yanok\Desktop\Data_Mining\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
# check the data
print("Existing Customers:")
existing_customers

Existing Customers:


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [18]:
print("\nPotential Customers:")
potential_customers


Potential Customers:


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
16277,64,NaN,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States
16278,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
16279,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


# Convert data into numerical data

In [ ]:
# split into X and y, 
# where X is the feature matrix
# and y is the column we want to predict


# split into training and test set

# Fill in the empty cells with Multivariate imputation

In [21]:
# notice we have missing values in the dataset
# to solve this we can make use of multivariate imputation from sklearn
# https://scikit-learn.org/stable/modules/impute.html
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(existing_customers)
IterativeImputer(random_state=0)
existing_customers = pd.DataFrame(imp.transform(existing_customers), columns=column_names)
potential_customers = pd.DataFrame(imp.transform(potential_customers), columns=column_names[:-1])


ValueError: could not convert string to float: 'State-gov'

In [19]:
existing_customers

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
potential_customers